In [1]:
magma_dir = '/home/ubuntu/magma/'
bucket_dir = '/home/ubuntu/s3/'
transformers_dir = '/home/ubuntu/transformers/'
cache_dir = bucket_dir+'.cache/'

## **Fine-tuning**

In [3]:
finetune_script = '"'+transformers_dir+'examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+transformers_dir+'examples/seq2seq/run_eval.py"'

### **Config**

In [4]:
import sys
sys.path.insert(0, magma_dir)
import config

import wandb
wandb.login()

project_name = 'ft_bart_para_wordembed_testw2v'
%env WANDB_PROJECT=$project_name

wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)


env: WANDB_PROJECT=ft_bart_para_wordembed_testw2v


### Karger Books Para Wordembed

In [5]:
model_name_or_path = 'sshleifer/distilbart-cnn-12-6'

In [6]:
data_dir = '"'+bucket_dir+'datasets/karger_books_para_wordembed/bart/st/"'

output_dir = '"'+bucket_dir+'fine-tuning/'+project_name+'"'

log_dir = output_dir + '/logs'

In [7]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.num_beams = 2

model_config.task_specific_params['summarization']['min_length'] = config.ONE_BULLET_MIN_LEN
model_config.task_specific_params['summarization']['max_length'] = config.ONE_BULLET_MAX_LEN
model_config.task_specific_params['summarization']['num_beams'] = 2
model_config_dir = '"'+bucket_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

##### Fine tune

In [8]:
'''
model_name_or_path = output_dir[:-1] + 'checkpoint-325"'
print(model_name_or_path)

from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6', use_cache=False)
tok.save_pretrained(model_name_or_path[1:-1])
'''

'\nmodel_name_or_path = output_dir[:-1] + \'checkpoint-325"\'\nprint(model_name_or_path)\n\nfrom transformers import AutoTokenizer\ntok = AutoTokenizer.from_pretrained(\'sshleifer/distilbart-cnn-12-6\', use_cache=False)\ntok.save_pretrained(model_name_or_path[1:-1])\n'

In [9]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--cache_dir $cache_dir \
--data_dir $data_dir \
--freeze_embeds \
--do_train \
--learning_rate 5e-5 \
--label_smoothing_factor 0.1 \
--warmup_steps 0 \
--fp16 \
--sortish_sampler \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--num_train_epochs 20 \
--logging_steps 10 --logging_first_step \
--per_device_train_batch_size 2 --per_device_eval_batch_size 8 \
--gradient_accumulation_steps 16 \
--evaluation_strategy steps --eval_steps 7 --eval_beams 2 \
--predict_with_generate \
--save_steps 98 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

02/15/2021 12:45:54 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
02/15/2021 12:45:54 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/home/marco/epfl/magma/fine-tuning/ft_bart_para_wordembed_testw2v', overwrite_output_dir=True, do_train=True, do_eval=None, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=20.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, logging_dir='runs/Feb15_12-45-54_calcolatore', logging_first_step=True, logging_steps=10, save_steps=98, save_total_limit=No

[INFO|modeling_utils.py:1143] 2021-02-15 12:46:14,747 >> All model checkpoint weights were used when initializing BartForConditionalGeneration.

[INFO|modeling_utils.py:1152] 2021-02-15 12:46:14,748 >> All the weights of BartForConditionalGeneration were initialized from the model checkpoint at sshleifer/distilbart-cnn-12-6.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.
02/15/2021 12:46:14 - INFO - utils -   setting model.config to task specific params for summarization:
 {'early_stopping': True, 'length_penalty': 2.0, 'max_length': 150, 'min_length': 10, 'no_repeat_ngram_size': 3, 'num_beams': 2}
02/15/2021 12:46:14 - INFO - utils -   note: command line args may override some of these
02/15/2021 12:46:15 - INFO - __main__ -   *** Train ***
/home/marco/miniconda3/envs/magma/lib/python3.6/site-packages/transformers/trainer.py:705: FutureWarning: `model_path` is de

of', 'intrathoracic', 'disease'], ['cigarette', 'addiction', 'involves', 'powerful', 'motivations', 'to', 'smoke', 'that', 'undermine', 'and', 'overwhelm', 'the', 'desire', 'to', 'avoid', 'smoking', 'because', 'of', 'its', 'social', 'financial', 'and', 'health', 'costs'], ['cigarette', 'addiction', 'stems', 'primarily', 'from', 'nicotine', 'dependence', 'repeated', 'rapid', 'intake', 'of', 'nicotine', 'from', 'cigarettes', 'sets', 'up', 'powerful', 'association', 'between', 'smoking', 'and', 'situations', 'in', 'which', 'smoking', 'typically', 'occurs', 'it', 'also', 'creates', 'nicotine', 'hunger', 'so', 'that', 'when', 'brain', 'nicotine', 'levels', 'fall', 'the', 'smoker', 'experiences', 'need', 'to', 'smoke', 'it', 'creates', 'unpleasant', 'withdrawal', 'symptoms', 'including', 'mood', 'disturbance', 'because', 'of', 'physiological', 'adaptation'], ['the', 'withdrawal', 'symptoms', 'are', 'relieved', 'by', 'smoking', 'thus', 'generating', 'the', 'feeling', 'of', 'need', 'to', 'smok